In [12]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from google.colab import drive

drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [13]:
device_name = tf.test.gpu_device_name()
if not device_name:
    raise SystemError('GPU device not found')
print(f"Found GPU at: {device_name}")

Found GPU at: /device:GPU:0


In [14]:
def get_datasets(train_dir, test_dir, img_size, batch_size):
    raw_train = keras.preprocessing.image_dataset_from_directory(
        train_dir,
        labels="inferred",
        label_mode="int",
        color_mode="grayscale",
        image_size=(img_size, img_size),
        batch_size=batch_size,
        shuffle=True,
        validation_split=0.2,
        subset="training",
        seed=123
    )
    raw_val = keras.preprocessing.image_dataset_from_directory(
        train_dir,
        labels="inferred",
        label_mode="int",
        color_mode="grayscale",
        image_size=(img_size, img_size),
        batch_size=batch_size,
        shuffle=True,
        validation_split=0.2,
        subset="validation",
        seed=123
    )
    raw_test = keras.preprocessing.image_dataset_from_directory(
        test_dir,
        labels="inferred",
        label_mode="int",
        color_mode="grayscale",
        image_size=(img_size, img_size),
        batch_size=batch_size,
        shuffle=False
    )

    class_names = raw_train.class_names

    norm = layers.Rescaling(1./255)
    AUTOTUNE = tf.data.AUTOTUNE

    train_ds = raw_train.map(lambda x,y:(norm(x),y)).cache().prefetch(AUTOTUNE)
    val_ds   = raw_val  .map(lambda x,y:(norm(x),y)).cache().prefetch(AUTOTUNE)
    test_ds  = raw_test .map(lambda x,y:(norm(x),y)).cache().prefetch(AUTOTUNE)

    return train_ds, val_ds, test_ds, class_names

In [15]:
def build_cnn(input_shape, num_classes):

    model = keras.Sequential([
        layers.Input(shape=input_shape),

        layers.Conv2D(32, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),

        layers.Conv2D(64, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),

        layers.Conv2D(128, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),

        layers.Flatten(),
        layers.Dense(256, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ])
    return model


In [22]:
def main():
    BASE_DIR  = "/content/drive/MyDrive/COMP562_Project/data/processed"
    TRAIN_DIR = os.path.join(BASE_DIR, "train")
    TEST_DIR  = os.path.join(BASE_DIR, "test")
    IMG_SIZE  = 224
    BATCH_SIZE= 32
    EPOCHS    = 100

    train_ds, val_ds, test_ds, class_names = get_datasets(
        TRAIN_DIR, TEST_DIR, IMG_SIZE, BATCH_SIZE
    )
    num_classes = len(class_names)
    print("Classes:", class_names)

    model = build_cnn((IMG_SIZE, IMG_SIZE, 1), num_classes)
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    model.summary()

    history = model.fit(
        train_ds,
        epochs=EPOCHS,
        validation_data=val_ds
    )

    test_loss, test_acc = model.evaluate(test_ds)
    print(f"Test accuracy: {test_acc:.4f}")

    return model, history

model, history = main()

Found 5715 files belonging to 4 classes.
Using 4572 files for training.
Found 5715 files belonging to 4 classes.
Using 1143 files for validation.
Found 1311 files belonging to 4 classes.
Classes: ['glioma', 'meningioma', 'notumor', 'pituitary']


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 224, 224, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 256)            │    25,690,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,784,068 (98.36 MB)

 Trainable params: 25,784,068 (98.36 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 27s 162ms/step - accuracy: 0.5025 - loss: 1.1815 - val_accuracy: 0.8110 - val_loss: 0.5930
Epoch 2/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 7s 48ms/step - accuracy: 0.7922 - loss: 0.5441 - val_accuracy: 0.8399 - val_loss: 0.4540
Epoch 3/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 7s 48ms/step - accuracy: 0.8606 - loss: 0.3898 - val_accuracy: 0.8486 - val_loss: 0.4168
Epoch 4/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 7s 48ms/step - accuracy: 0.8743 - loss: 0.3229 - val_accuracy: 0.8565 - val_loss: 0.4040
Epoch 5/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 7s 48ms/step - accuracy: 0.9125 - loss: 0.2369 - val_accuracy: 0.8574 - val_loss: 0.4230
Epoch 6/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 7s 49ms/step - accuracy: 0.9232 - loss: 0.2050 - val_accuracy: 0.8933 - val_loss: 0.4274
Epoch 7/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 10s 47ms/step - accuracy: 0.9477 - loss: 0.1469 - val_accuracy: 0.8959 - val_loss: 0.3879
Epoch 8/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 10s 47ms/step - accuracy: 0.9605 - loss: 0.107